In [2]:
import random
from math import floor, exp, sqrt, pi, cos, sin

In [1]:
# Установка библиотеки sklearn
!pip3 install sklearn


[notice] A new release of pip available: 22.3 -> 23.1
[notice] To update, run: C:\Users\sasha\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [3]:
from sklearn.linear_model import LinearRegression

ModuleNotFoundError: No module named 'sklearn'

In [4]:
def round_to_2(x):
    """
    Принимает число и возвращает результат его округления
    до 2 знаков после запятой.
    
    Аргументы:
        x: Число.
        
    Возвращаемое значение:
        Результат округления числа до 2 знаков после запятой.
    """
    
    return round(x, 2)

# Grid search

In [ ]:
def score_model(model, x_val, y_val):
    """
    Оценивает точность модели по метрике «среднее отклонение от предсказанного значения».
    
    Аргументы:
        model: Модель.
        x_test: Список объектов тестовой выборки.
        y_test: Список значений предсказываемой характеристики для объектов из тестовой выборки.
                Значение на $i$-й позиции в списке соответствует $i$-му объекту тестовой выборки.
        
    Возвращаемое значение:
        Возвращает точность модели.
    """
    
    y_pred = model.predict(x_val)
    
    res = 0

    for i in range(len(y_val)):
        res += abs(y_pred[i] - y_val[i])
        
    return res / len(y_val)

In [ ]:
def grid_search_solution(model, 
                         model_configs,
                         x_train, y_train, 
                         x_val, y_val):
    """
    Принимает набор конфигураций модели линейной регрессии и находит лучшую из них
    с помощью алгоритма grid search.
    Для оценки точности модели на валидационной выборке используется функция score_model.
    
    Аргументы:
        model: Модель.
        model_configs: Список конфигураций модели линейной регрессии.
                       Каждая конфигурация — список номеров факторов, 
                       которые используются для обучения модели.
        x_train: Список объектов обучающей выборки.
        y_train: Список значений предсказываемой характеристики для объектов из обучающей выборки.
                 Значение на $i$-й позиции в списке соответствует $i$-му объекту обучающей выборки.
        x_val: Список объектов валидационной выборки.
        y_val: Список значений предсказываемой характеристики для объектов из валидационной выборки.
               Значение на $i$-й позиции в списке соответствует $i$-му объекту валидационной выборки.
        
    Возвращаемое значение:
        Возвращает пару значений: лучшая конфигурация, точность модели с данной конфигурацией 
        на валидационной выборке.
    """
    
    #a = [[4], [5], [6]]
    #b = [4, 5, 6]
    #print(score_model(model, a,b))
    train = []
    val = []
    ans_score = -1


    print(len(x_train))
    print(x_train[0])
    print(len(x_val))
    print(x_val[0])

    for mdl in model_configs:
        for x in mdl:
            for xtr in x_train:
                train.append([xtr[x]])
            for xvl in x_val:
                val.append([xvl[x]])
        #print(len(train), len(y_train))
            model.fit(train, y_train)
            print(len(train), len(y_train))
        #print(train,y_train,  val, y_val)
            score = round_to_2(score_model(model, val, y_val))
        #print(round_to_2(score_model(model, val, y_val)))
            train.clear()
            val.clear()
        if score < ans_score or ans_score == -1:
            ans_score = score
            ans = mdl

    print(ans, ans_score)
    return ans, ans_score

In [ ]:
def process_data(file_path, frac=0.9):
    data_x_y = []
    
    with open(file_path) as f:
        cnt = 0
        
        for line in f:
            if cnt == 0:
                cnt += 1
                continue
                
            l = line.strip().split(',')
            data_x_y.append(([float(x) for x in l[:-1]], float(l[-1])))  
            
            cnt += 1
            
    cut_n = floor(frac * len(data_x_y))
            
    train_x_y = data_x_y[:cut_n]
    val_x_y = data_x_y[cut_n:]
    
    train_x, train_y = [], []
    
    for x, y in train_x_y:
        train_x.append(x)
        train_y.append(y)
        
    val_x, val_y = [], []
    
    for x, y in val_x_y:
        val_x.append(x)
        val_y.append(y)
    
    return train_x, train_y, val_x, val_y

def grid_search_test():
    def split_df_to_xs_and_ys(df):
        xs = []
        ys = []
        
        for _, row in df.iterrows():
            xs.append([row['x1'], row['x2'], row['x3'], row['x4']])
            ys.append(row['y'])
            
        return xs, ys
        
    data_x_train_example_1 = [[1, -33], [2, -21], [3, -34234]]
    data_y_train_example_1 = [1, 2, 3]
    
    data_x_val_example_1 = [[4, -231], [5, -342341], [6, -23]]
    data_y_val_example_1 = [4, 5, 6]
    
    configs_example_1 = [[0], [1]]
    
    res_example_1 = ([0], 0)
    
    assert grid_search_solution(LinearRegression(),
                                configs_example_1,
                                data_x_train_example_1, data_y_train_example_1,
                                data_x_val_example_1, data_y_val_example_1) == res_example_1
    
    data_x_train_example_2, data_y_train_example_2, \
        data_x_val_example_2, data_y_val_example_2 = process_data('grid_search_test_data.csv', frac=0.9)
    
    configs_example_2 = [[0], [0, 1], [0, 1, 2], [2, 3], [1]]
    
    res_example_2 = ([0, 1, 2], 0.26)
    
    assert grid_search_solution(LinearRegression(),
                                configs_example_2,
                                data_x_train_example_2, data_y_train_example_2,
                                data_x_val_example_2, data_y_val_example_2) == res_example_2

    print('Тест прошёл успешно!')

In [ ]:
grid_search_test()

# Метод дихотомии

In [35]:
def dichotomous_search_solution(f, a, b, eps):
    """
    Производит поиск минимума заданной функции в заданном интервале с помощью метода дихотомии.
    
    Аргументы:
        f: Функция от одного аргумента, минимум которой необходимо найти.
        a: Левая граница интервала, в котором происходит поиск минимума.
        b: Правая граница интервала, в котором происходит поиск минимума.
        eps: Допустимая погрешность при поиске минимума.
        
    Возвращаемое значение:
        Возвращает координату точки, в которой достигается минимальное значение функции.
        Координата должна быть округлена до 2 знаков после запятой.
    """
    
    while(b - a > 2 * eps):
        if f((a + b - eps) / 2) > f((a + b + eps) / 2):
            a = (a + b - eps) / 2
        else:
            b = (a + b + eps) / 2
        
    return round_to_2((a + b) / 2)

In [36]:
def dichotomous_search_test():
    f_example_1 = lambda x: x ** 2
    
    a_example_1 = -2
    b_example_1 = 5
    eps_example_1 = 0.01
    
    res_example_1 = 0
    
    min_example_1 = dichotomous_search_solution(f_example_1, a_example_1, b_example_1, eps_example_1)
    
    if min_example_1 == 0.0:
        min_example_1 = abs(min_example_1)
    
    assert min_example_1 == res_example_1
    
    f_example_2 = lambda x: (exp(2 * (x - 2)) + 1) * (x + 1) ** 2
    
    a_example_2 = -100
    b_example_2 = 200
    eps_example_2 = 0.01
    
    res_example_2 = -1
    
    min_example_2 = dichotomous_search_solution(f_example_2, a_example_2, b_example_2, eps_example_2)
    
    if min_example_2 == 0.0:
        min_example_2 = abs(min_example_2)
    
    assert min_example_2 == res_example_2

    print('Тест прошёл успешно!')

In [37]:
dichotomous_search_test()

Тест прошёл успешно!


# Градиентный спуск

In [80]:
def gradient_descent_solution(grad_f, x_0, alpha, eps):
    """
    Производит поиск минимума заданной функции двух аргументов с помощью градиентного спуска.
    
    Аргументы:
        grad_f: Градиент функции двух аргументов, для которой необходимо найти минимум.
                Функция принимает на вход точку (список из двух значений) и возвращает
                вектор градиента в этой точке (тоже список из двух значений).
        x_0: Стартовая точка (список из двух значений), из которой запускается алгоритм градиентного спуска.
        alpha: Коэффициент скорости обучения.
        eps: Минимальное расстояние между текущей точкой градиентного спуска и следующей,
             при котором работа алгоритма останавливается.
        
    Возвращаемое значение:
        Координаты точки (список из двух значений), в которой достигается минимальное значение функции.
        Каждая координата должна быть округлена до 2 знаков после запятой.
    """
    dist_v = lambda x1, x2: ((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2) ** (1/2)
    alp = lambda a, x: [a * x[0], a*x[1]]
    minus = lambda x1, x2: [x1[0] - x2[0], x1[1] - x2[1]]
    
    points = []
    points.append(x_0)
    points.append(minus(x_0, alp(alpha, grad_f(x_0))))
    
    i = 1
    
    while dist_v(points[i], points[i-1]) > eps:
        points.append(minus(points[i], alp(alpha, grad_f(points[i]))))
        i += 1
    points[len(points) - 1][0] = round_to_2(points[len(points) - 1][0])
    points[len(points) - 1][1] = round_to_2(points[len(points) - 1][1])
    
    return points[len(points) - 1]
    

In [81]:
def gradient_descent_test():
    f_example_1 = lambda x: x[0] ** 4 + x[1] ** 4
    grad_f_example_1 = lambda x: [4 * x[0], 4 * x[1]]
    
    x_0_example_1 = [32, -4]
    
    alpha_example_1 = 0.01
    eps_example_1 = 0.001
    
    res_example_1 = [0.02, 0]
    
    assert gradient_descent_solution(grad_f_example_1, x_0_example_1, alpha_example_1, eps_example_1) == res_example_1
    
    f_example_2 = lambda x: (x[0] - 1) ** 4 + 2 * (x[1] + 2) ** 2 - 1
    grad_f_example_2 = lambda x: [4 * (x[0] - 1) ** 3, 4 * (x[1] + 2)]
    
    x_0_example_2 = [-2, 2]
    
    alpha_example_2 = 0.0001
    eps_example_2 = 0.0001
    
    res_example_2 = [0.58, -1.76]
    
    assert gradient_descent_solution(grad_f_example_2, x_0_example_2, alpha_example_2, eps_example_2) == res_example_2

    print('Тест прошёл успешно!')

In [82]:
gradient_descent_test()

[0.02, -0.0]
[0.58, -1.76]
Тест прошёл успешно!


# Метод имитации отжига

In [ ]:
def g(x):
    return 2 * x ** 2 + cos(pi * x) - 2.9 * sin(2 * pi * x) + cos(3 * pi * x) * sin(pi * x)

In [83]:
def sub_with_p(random_gen, p, new_x, x):
    """
    С заданной вероятностью возвращает точку `new_x`, в остальных случаях
    возвращает точку x.
    
    Аргументы:
        random_gen: Генератор случайных чисел.
        p: Вероятность, с которой нужно вернуть первую точку.
        new_x: Точка, которая возвращается с вероятностью p.
        x: Точка, которая возвращает в остальных случаях.
        
    Возвращаемое значение:
        Точка, которая с вероятностью p будет равна new_x. В противном случае точка будет равна x.
    """
    
    val = random_gen.random()
    
    if val > p:
        return x
    
    return new_x

In [114]:
def simulated_annealing_solution(f, x_0, t_0, lam, eps, random_gen):
    """
    Производит поиск минимума функции с помощью метода имитации отжига.
    
    Аргументы:
        f: Функция, минимум которой необходимо найти.
        x_0: Стартовая точка, из которой начинается процесс поиска минимума.
        t_0: Начальная температура системы.
        lam: Коэффициент охлаждения системы на каждом шаге.
             Охлаждение происходит по правилу T = lam * T.
        eps: Когда температура становится меньше eps, метод имитации отжига останавливает свою работу.
        random_gen: Генератор случайных чисел.
        
    Возвращаемое значение:
        Точка, которую метод считает минимумом функции.
    """
    p = []
    p.append(x_0)
    
    i = 0
    while t_0 > eps:
        t_0 = t_0 * lam
        x = p[i]
        x_ = x + random.uniform(-1.0, 1.0)
        print(x_)
        delt = f(x_) - f(x)
        
        if delt > 0:
            #print(round_to_2(eps ** -(delt/ t_0)))
            sub_with_p(random_gen, eps ** -(delt/ t_0), x_, x)
        else:
            p.append(x_)
            
    return round_to_2(p[len(p) - 1])

In [115]:
def simulated_annealing_test():
    weierstrass_function = lambda x: 10 + x ** 2 - 10 * cos(2 * pi * x)

    f_example_1 = weierstrass_function
    
    x_0_example_1 = 20
    t_0_example_1 = 1000
    lam_example_1 = 0.99
    eps_example_1 = 0.001
    
    random_gen_example_1 = random.Random(0)
    
    res_example_1 = 0.0
    
    found_min_example_1 = \
        simulated_annealing_solution(weierstrass_function,
                                     x_0_example_1,
                                     t_0_example_1,
                                     lam_example_1,
                                     eps_example_1,
                                     random_gen_example_1)
    
    assert round_to_2(weierstrass_function(found_min_example_1)) == res_example_1
    
    print('Тест прошёл успешно!')

In [116]:
simulated_annealing_test()

20.043876184038616
19.39795412050353
20.642599776519695
19.78676276640039
19.880119741014717
19.955655702097193
19.84732435359143
20.22303937290067
19.500003960621004
20.582503668445504
19.44348520782911
20.24711761021643
20.43099237640417
20.0302673843879
19.003744935563564
20.869095399237025
19.22605608761225
20.109611639730367
20.239873489892027
19.569589278697514
20.34407667005333
20.55275831289275
19.111264090480997
19.435610202861582
20.592878640018725
20.779942916653177
20.16032236195073
19.70005570887273
20.663380485090087
20.177257707311654
20.97912763332914
19.476580813765008
20.824349082505393
19.0865225426656
19.407030445817306
19.299764510013294
20.40413732339676
20.301036892033075
20.130216088857473
19.038317640780672
19.63620308906414
19.76424885517144
19.92385995309042
20.842481093383967
19.88130642241255
19.506244230187296
20.531724272380092
19.404974212805858
19.13133069091624
19.481455288586876
19.76920040850794
19.2751256664757
19.884230148482658
19.515999528798027


OverflowError: (34, 'Result too large')